# Creating a Sampled Dataset

**Learning Objectives**
- Sample the natality dataset to create train/eval/test sets
- Preprocess the data in Pandas dataframe

## Introduction

In this notebook we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe. 

In [1]:
PROJECT = "cloud-training-demos"  # Replace with your PROJECT
BUCKET = "cloud-training-bucket"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.13"                # TF version for CMLE to use

In [2]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets.

In [4]:
# Create SQL query using natality data after the year 2000
query_string = """
WITH
  CTE_hash_cols_fixed AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
      WHEN day IS NULL AND wday IS NULL THEN 0
    ELSE
    CASE
      WHEN day IS NULL THEN wday
    ELSE
    wday
  END
  END
    AS date,
    IFNULL(state,
      "Unknown") AS state,
    IFNULL(mother_birth_state,
      "Unknown") AS mother_birth_state
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000)

SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING), CAST(date AS STRING), CAST(state AS STRING), CAST(mother_birth_state AS STRING)))) AS hashvalues
FROM
  CTE_hash_cols_fixed
"""

There are only a limited number of years, months, days, and states in the dataset. Let's see what the hash values are.

We'll call BigQuery but group by the hashcolumn and see the number of records for each group. This will enable us to get the correct train/eval/test percentages

In [5]:
from google.cloud import bigquery
bq = bigquery.Client(project = PROJECT)

df = bq.query("SELECT hashvalues, COUNT(weight_pounds) AS num_babies FROM (" 
              + query_string + 
              ") GROUP BY hashvalues").to_dataframe()

print("There are {} unique hashvalues.".format(len(df)))
df.head()

There are 658107 unique hashvalues.


,hashvalues,num_babies
0,1557970813427928760,380
1,8183651910314772965,857
2,3499106308324393870,236
3,588280865661544184,282
4,6927803069586192700,55


We can make a query to check if our bucketing values result in the correct sizes of each of our dataset splits and then adjust accordingly

In [6]:
sampling_percentages_query = """
WITH
  -- Get label, features, and column that we are going to use to split into buckets on
  CTE_hash_cols_fixed AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
      WHEN day IS NULL AND wday IS NULL THEN 0
    ELSE
    CASE
      WHEN day IS NULL THEN wday
    ELSE
    wday
  END
  END
    AS date,
    IFNULL(state,
      "Unknown") AS state,
    IFNULL(mother_birth_state,
      "Unknown") AS mother_birth_state
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000),
  CTE_data AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING), CAST(date AS STRING), CAST(state AS STRING), CAST(mother_birth_state AS STRING)))) AS hashvalues
  FROM
    CTE_hash_cols_fixed),
  -- Get the counts of each of the unique hashs of our splitting column
  CTE_first_bucketing AS (
  SELECT
    hashvalues,
    COUNT(*) AS num_records
  FROM
    CTE_data
  GROUP BY
    hashvalues ),
  -- Get the number of records in each of the hash buckets
  CTE_second_bucketing AS (
  SELECT
    MOD(hashvalues, {0}) AS bucket_index,
    SUM(num_records) AS num_records
  FROM
    CTE_first_bucketing
  GROUP BY
    MOD(hashvalues, {0})),
  -- Calculate the overall percentages
  CTE_percentages AS (
  SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
      SUM(num_records)
    FROM
      CTE_second_bucketing) AS percent_records
  FROM
    CTE_second_bucketing ),
  -- Choose which of the hash buckets will be used for training and pull in their statistics
  CTE_train AS (
  SELECT
    *,
    "train" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= 0
    AND bucket_index < {1}),
  -- Choose which of the hash buckets will be used for validation and pull in their statistics
  CTE_eval AS (
  SELECT
    *,
    "eval" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {1}
    AND bucket_index < {2}),
  -- Choose which of the hash buckets will be used for testing and pull in their statistics
  CTE_test AS (
  SELECT
    *,
    "test" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {2}
    AND bucket_index < {0}),
  -- Union the training, validation, and testing dataset statistics
  CTE_union AS (
  SELECT
    0 AS dataset_id,
    *
  FROM
    CTE_train
  UNION ALL
  SELECT
    1 AS dataset_id,
    *
  FROM
    CTE_eval
  UNION ALL
  SELECT
    2 AS dataset_id,
    *
  FROM
    CTE_test ),
  -- Show final splitting and associated statistics
  CTE_split AS (
  SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
  FROM
    CTE_union
  GROUP BY
    dataset_id,
    dataset_name )
SELECT
  *
FROM
  CTE_split
ORDER BY
    dataset_id
"""

modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

df = bq.query(sampling_percentages_query.format(modulo_divisor, train_buckets, train_buckets + eval_buckets)).to_dataframe()
df.head()

,dataset_id,dataset_name,num_records,percent_records
0,0,train,26080035,0.783845
1,1,eval,3639721,0.109393
2,2,test,3552158,0.106761


Here's a way to get a well-distributed portion of the data in such a way that the train/eval/test sets do not overlap. 

In [7]:
# Added every_n so that we can now subsample from each of the hash values to get approximately the record counts we want
every_n = 500

train_query = "SELECT * FROM ({0}) WHERE MOD(hashvalues, {1} * 100) < 80".format(query_string, every_n)
eval_query = "SELECT * FROM ({0}) WHERE MOD(hashvalues, {1} * 100) >= 80 AND MOD(hashvalues, {1} * 100) < 90".format(query_string, every_n)
test_query = "SELECT * FROM ({0}) WHERE MOD(hashvalues, {1} * 100) >= 90 AND MOD(hashvalues, {1} * 100) < 100".format(query_string, every_n)

train_df = bq.query(train_query).to_dataframe()
eval_df = bq.query(eval_query).to_dataframe()
test_df = bq.query(test_query).to_dataframe()

print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 18998 examples in the train dataset.
There are 1513 examples in the validation dataset.
There are 1389 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`. Also, if there is more than child we'll change the `plurality` to `Multiple(2+)`. While we're at it, We'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [8]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
0,8.483388,True,42,1,39.0,6380000856187650041
1,6.812284,True,44,2,36.0,7823092823372300071
2,8.234265,True,42,1,38.0,1861629504184950069
3,7.275255,True,42,1,38.0,5734139609924950008
4,4.640731,False,51,2,40.0,3838362205187550017


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [9]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashvalues
count,18978.000000,18998.000000,18998.000000,18850.000000,1.899800e+04
mean,7.243111,27.782346,1.038162,38.639735,4.288488e+18
std,1.337393,6.195585,0.210196,2.594170,2.534495e+18
min,0.500449,13.000000,1.000000,17.000000,5.826385e+15
25%,6.572531,23.000000,1.000000,38.000000,1.861630e+18
50%,7.339189,28.000000,1.000000,39.000000,3.835475e+18
75%,8.062305,32.000000,1.000000,40.000000,6.784884e+18
max,12.500210,51.000000,4.000000,47.000000,9.210618e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

In [10]:
import pandas as pd

def preprocess(df):
    # Clean up data
    # Remove what we don"t want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace = True)

    # Now create extra rows to simulate lack of ultrasound
    no_ultrasound = df.copy(deep = True)
    no_ultrasound.loc[no_ultrasound["plurality"] != "Single(1)", "plurality"] = "Multiple(2+)"
    no_ultrasound["is_male"] = "Unknown"

    # Concatenate both datasets together and shuffle
    return pd.concat([df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

Let's process the train/eval/test set and see a small sample of the training data after our preprocessing:

In [11]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [12]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
0,10.430070,Unknown,34,Single(1),40.0,2644101647865250034
1,5.687926,False,23,Single(1),38.0,701411312121250028
2,8.741329,False,22,Single(1),41.0,1861629504184950069
3,3.875727,Unknown,18,Single(1),36.0,6300252072602950014
4,6.635914,False,29,Single(1),38.0,6784884401981100070


In [13]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
37671,6.541115,False,23,Single(1),38.0,2177280183356650077
37672,7.881526,Unknown,30,Single(1),39.0,3480083103445950001
37673,8.476774,Unknown,20,Single(1),40.0,4614303140002600076
37674,7.125340,Unknown,23,Single(1),40.0,7524082756449550067
37675,7.771295,Unknown,31,Single(1),40.0,3480083103445950001


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [14]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hashvalues
count,37676.000000,37676.000000,37676.000000,3.767600e+04
mean,7.244695,27.767544,38.644973,4.277547e+18
std,1.335645,6.189406,2.576290,2.535502e+18
min,0.500449,13.000000,18.000000,5.826385e+15
25%,6.580799,23.000000,38.000000,1.861630e+18
50%,7.341393,28.000000,39.000000,3.825089e+18
75%,8.062305,32.000000,40.000000,6.784884e+18
max,12.500210,51.000000,47.000000,9.210618e+18


## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

In [15]:
columns = "weight_pounds,is_male,mother_age,plurality,gestation_weeks".split(',')
train_df.to_csv(path_or_buf = "train.csv", columns = columns, header = False, index = False)
eval_df.to_csv(path_or_buf = "eval.csv", columns = columns, header = False, index = False)
test_df.to_csv(path_or_buf = "test.csv", columns = columns, header = False, index = False)

In [16]:
%%bash
wc -l *.csv

   3000 eval.csv
   2764 test.csv
  37676 train.csv
  43440 total


In [17]:
%%bash
head *.csv

==> eval.csv <==
6.4374980503999994,False,21,Single(1),37.0
10.06189763768,Unknown,23,Single(1),40.0
6.686620406459999,False,23,Single(1),40.0
7.936641432,False,36,Single(1),38.0
7.0437692708999995,Unknown,37,Single(1),39.0
7.62578964258,False,31,Single(1),39.0
6.1883756943399995,True,25,Single(1),38.0
8.3334735036,True,21,Single(1),38.0
7.81318256528,True,29,Single(1),38.0
8.18796841068,False,20,Single(1),39.0

==> test.csv <==
8.50102482272,Unknown,35,Single(1),40.0
9.4909003791,True,26,Single(1),40.0
8.18796841068,Unknown,23,Single(1),37.0
7.87491199864,Unknown,39,Single(1),38.0
7.3083239852999995,True,30,Single(1),38.0
7.17825125072,Unknown,29,Single(1),38.0
6.1244416383599996,False,22,Single(1),38.0
6.6579603124,True,23,Single(1),39.0
8.000575487979999,Unknown,40,Single(1),38.0
6.77480531126,Unknown,21,Single(1),39.0

==> train.csv <==
10.430069615219999,Unknown,34,Single(1),40.0
5.6879263596,False,23,Single(1),38.0
8.7413286883,False,22,Single(1),41.0
3.87572656596,Unknown,18,Sin

In [18]:
%%bash
tail *.csv

==> eval.csv <==
7.31273323054,Unknown,21,Single(1),43.0
7.7492485093,False,31,Single(1),47.0
9.43798943622,False,25,Single(1),39.0
6.1244416383599996,False,29,Single(1),40.0
12.125424409999999,True,21,Single(1),39.0
8.0138032237,Unknown,21,Single(1),41.0
8.62448368944,False,20,Single(1),39.0
8.56275425608,Unknown,35,Single(1),39.0
7.7933409617,Unknown,36,Single(1),41.0
1.5542589471,Unknown,31,Single(1),26.0

==> test.csv <==
7.25100379718,Unknown,16,Single(1),41.0
9.12493302418,True,23,Single(1),41.0
8.375361333379999,False,39,Single(1),39.0
6.8122838958,Unknown,40,Single(1),39.0
6.4815905028,True,21,Single(1),37.0
7.43839671988,Unknown,26,Single(1),39.0
7.6279942652,True,32,Single(1),39.0
8.313631900019999,Unknown,43,Single(1),39.0
6.9996768185,Unknown,22,Single(1),37.0
8.27615331548,Unknown,35,Single(1),41.0

==> train.csv <==
8.9948602896,Unknown,37,Single(1),43.0
8.93754010148,True,25,Single(1),40.0
3.12615487516,Unknown,34,Multiple(2+),38.0
6.93794738514,Unknown,33,Single(1),38.0

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License